In [1]:
import gmql as gl

In [2]:
# loading of a dataset
data_path = "/home/luca/Documenti/resources/hg_narrowPeaks"
output_path = "/home/luca/Documenti/resources/result"

parser = gl.parsers.NarrowPeakParser()

data = gl.GMQLDataset(parser=parser)\
                .load_from_path(path=data_path, meta_load=False)
    
# selects all the data coming from cell K562 and having antibody H3K4me3
condition = (data.MetaField("cell") == 'K562') & (data.MetaField("antibody") == 'H3K4me3')
filtered_data = data.meta_select(predicate=condition) # test

In [3]:
# project on region fields
new_reg_field = {
    "new_attribute" : (filtered_data.RegField("pValue") + filtered_data.RegField("signalValue"))/2
}

In [4]:
proj_data = filtered_data.reg_project(field_list=['score', 'signalValue', 'pValue'], new_field_dict=new_reg_field)

In [5]:
# cover operation

new_fields = {
    "min_pValue" : gl.MIN("pValue")
}
cover_data = proj_data.normal_cover(minAcc=1, maxAcc=2, groupBy=['treatment_tag'], new_reg_fields=new_fields)

In [6]:
# materialization
cover_data = cover_data.materialize(output_path=output_path)

2017-04-25 21:03:55,992 - PyGML logger - INFO - Loading meta data from path /home/luca/Documenti/resources/result/exp/


100%|██████████| 6/6 [00:00<00:00, 638.52it/s]

2017-04-25 21:03:56,052 - PyGML logger - INFO - dataframe construction



100%|██████████| 94/94 [00:00<00:00, 197.62it/s]

2017-04-25 21:03:56,539 - PyGML logger - INFO - Loading region data from path /home/luca/Documenti/resources/result/exp/



100%|██████████| 8/8 [00:06<00:00,  1.18it/s]


In [8]:
cover_data.regs

,AccIndex,JaccardIntersect,JaccardResult,chr,min_pValue,start,stop,strand
id_sample,,,,,,,,
3671497045303514216,1,1.0,1.000000,chr1,3.80610,118520,118670,*
3671497045303514216,2,1.0,1.000000,chr1,63.91700,137700,137850,*
3671497045303514216,2,1.0,1.000000,chr1,86.76090,138420,138570,*
3671497045303514216,2,1.0,1.000000,chr1,86.76090,138680,138830,*
3671497045303514216,2,1.0,0.764706,chr1,61.09850,138940,139110,*
3671497045303514216,2,1.0,0.578947,chr1,82.22120,139320,139510,*
3671497045303514216,1,1.0,1.000000,chr1,7.16405,229540,229690,*
3671497045303514216,1,1.0,1.000000,chr1,3.91562,229940,230090,*
3671497045303514216,1,1.0,1.000000,chr1,6.42974,235580,235730,*
